In [1]:
%run utils/devtools.py

In [56]:
%reloadmypkg utils

import json
import os
from tqdm import tqdm
from utils.url2platform import problem_url2platform
from utils.standardise_url import expand_url, strip_query_params
from utils.parse4db import problems_parse4db, tutorials_parse4db
from utils.enums import ProblemPlatform

✅ Reloaded package 'utils' and its submodules.


In [3]:
sheet_path = "../raw-data/striver/a2z-sheet-response-data.json"
export_file = "../cleaned-data/striver/a2z.json"

In [4]:
with open(sheet_path) as file:
        raw = json.load(file)
problems_data = [
        {"gfg": topic["gfg_link"], "c360": topic["cs_link"], "lc": topic["lc_link"], "title": topic["question_title"], "yt": topic["yt_link"], "post": topic["post_link"]}
        for step in raw["sheetData"]
        for substep in step["sub_steps"]
        for topic in substep["topics"]
]

In [5]:
urls_titles = []
for data in problems_data:
    url = ""
    if data["lc"]:
        url = data["lc"]
    elif data["gfg"]:
        url = data["gfg"]
    elif data["c360"]:
        url = data["c360"]
    else:
        url = data["yt"] if data["yt"] is not None else data["post"]
    urls_titles.append({"url": url, "title": data["title"]})

In [6]:
len(urls_titles)

455

In [7]:
urls_titles[:3]

[{'url': 'https://practice.geeksforgeeks.org/problems/search-query-auto-complete/0?category%5B%5D=Strings&category%5B%5D=Strings&problemStatus=unsolved&difficulty%5B%5D=2&page=1&query=category%5B%5DStringsproblemStatusunsolveddifficulty%5B%5D2page1category%5B%5DStr',
  'title': 'User Input / Output'},
 {'url': 'https://bit.ly/3Dqmgvx', 'title': 'Data Types'},
 {'url': 'https://practice.geeksforgeeks.org/problems/java-if-else-decision-making0924/0?category%5B%5D=Java&category%5B%5D=Java&difficulty%5B%5D=-2&page=1&query=category%5B%5DJavadifficulty%5B%5D-2page1category%5B%5DJava',
  'title': 'If Else statements'}]

In [8]:
for index, item in enumerate(urls_titles):
    if  item["url"] is None:
        print(f"{index}: {item}")

11: {'url': None, 'title': 'Java Collections'}


In [9]:
# Adding link for Java Collections as it is missing
urls_titles[11]["url"] = "https://www.youtube.com/playlist?list=PLA3GkZPtsafZZsLj0Tybu3y0HVl-hp1ea"

urls_titles[11]

{'url': 'https://www.youtube.com/playlist?list=PLA3GkZPtsafZZsLj0Tybu3y0HVl-hp1ea',
 'title': 'Java Collections'}

In [10]:
for index, item in enumerate(urls_titles):
    if "https://leetcode.com/accounts/login/?next=/problems/find-the-celebrity/" in item["url"]:
        print(f"{index}: {item}")

226: {'url': 'https://leetcode.com/accounts/login/?next=/problems/find-the-celebrity/', 'title': 'The Celebrity Problem'}


In [11]:
# Replace premium LeetCode problem with LintCode link
urls_titles[226]["url"] = "https://www.lintcode.com/problem/645/"

urls_titles[226]

{'url': 'https://www.lintcode.com/problem/645/',
 'title': 'The Celebrity Problem'}

In [12]:
for index, item in enumerate(urls_titles):
    if "implement-strstr" in item["url"]:
        print(f"{index}: {item}")

450: {'url': 'https://leetcode.com/problems/implement-strstr/', 'title': 'Z-Function'}
451: {'url': 'https://leetcode.com/problems/implement-strstr/', 'title': 'KMP algo / LPS(pi) array'}


In [ ]:
# Fix the redirect urls
urls_titles[450]["url"] = "https://leetcode.com/problems/find-the-index-of-the-first-occurrence-in-a-string/"
urls_titles[451]["url"] = "https://leetcode.com/problems/find-the-index-of-the-first-occurrence-in-a-string/"

In [25]:
for index, item in enumerate(urls_titles):
    if "coin-change-2" in item["url"]:
        print(f"{index}: {item}")

404: {'url': 'https://leetcode.com/problems/coin-change-2/', 'title': 'Coin Change 2 (DP - 22)'}


In [26]:
# Fix the redirect urls
urls_titles[404] = "https://leetcode.com/problems/coin-change-ii/"

In [27]:
urls_titles[404]

'https://leetcode.com/problems/coin-change-ii/'

In [14]:
print(urls_titles[450])
print(urls_titles[451])

{'url': 'https://leetcode.com/problems/find-the-index-of-the-first-occurrence-in-a-string/', 'title': 'Z-Function'}
{'url': 'https://leetcode.com/problems/find-the-index-of-the-first-occurrence-in-a-string/', 'title': 'KMP algo / LPS(pi) array'}


In [15]:
problems_urls_titles = []
tutorials_urls_titles = []
for index in tqdm(range(len(urls_titles)), desc="Expanding SDE URLS", ncols=100):
    item = urls_titles[index]
    url = item["url"]
    title = item["title"]
    if "bit.ly" in url or "codingninjas.com/codestudio" in url or "codingninjas.com/studio" in url:
        url = expand_url(url)
    platform = problem_url2platform(url)
    if platform != ProblemPlatform.UNKNOWN:
        url = strip_query_params(url)
        problems_urls_titles.append({"url": url, "title": title})
    else:
        tutorials_urls_titles.append({"url": url, "title": title})

Expanding SDE URLS:   0%|▏                                          | 2/455 [00:01<06:56,  1.09it/s]

Unknown platform for url: https://youtu.be/EAR7De6Goz4?t=2415


Expanding SDE URLS:   2%|▊                                          | 8/455 [00:04<04:10,  1.79it/s]

Unknown platform for url: https://youtu.be/FPu9Uld7W-E
Unknown platform for url: https://www.youtube.com/watch?v=tNm_NNSB3_w&list=PLgUwDviBIf0oF6QL8m22w1hIDC1vJ_BHz&index=3
Unknown platform for url: https://www.youtube.com/watch?v=RRVYpIET_RU
Unknown platform for url: https://www.youtube.com/playlist?list=PLA3GkZPtsafZZsLj0Tybu3y0HVl-hp1ea


Expanding SDE URLS:   5%|██                                        | 23/455 [00:08<02:49,  2.54it/s]

...

Expanding SDE URLS:  36%|██████████████▌                          | 162/455 [01:22<03:12,  1.52it/s]

Unknown platform for url: https://www.geeksforgeeks.org/generate-binary-strings-without-consecutive-1s/?utm_source=youtube&utm_medium=collab_striver_ytdescription&utm_campaign=generate-binary-strings-without-consecutive-1s


Expanding SDE URLS:  56%|██████████████████████▉                  | 254/455 [02:08<02:25,  1.38it/s]

Unknown platform for url: https://www.geeksforgeeks.org/find-minimum-number-of-coins-that-make-a-change/


Expanding SDE URLS: 100%|█████████████████████████████████████████| 455/455 [04:00<00:00,  1.89it/s]


In [16]:
len(problems_urls_titles)

447

In [17]:
problems_urls_titles[:5]

[{'url': 'https://practice.geeksforgeeks.org/problems/search-query-auto-complete/0',
  'title': 'User Input / Output'},
 {'url': 'https://www.geeksforgeeks.org/problems/data-type-1666706751/1',
  'title': 'Data Types'},
 {'url': 'https://practice.geeksforgeeks.org/problems/java-if-else-decision-making0924/0',
  'title': 'If Else statements'},
 {'url': 'https://practice.geeksforgeeks.org/problems/java-switch-case-statement3529/1',
  'title': 'Switch Statement'},
 {'url': 'https://www.naukri.com/code360/problems/nth-fibonacci-number_74156',
  'title': 'For loops'}]

In [18]:
len(tutorials_urls_titles)

8

In [19]:
tutorials_urls_titles

[{'url': 'https://youtu.be/EAR7De6Goz4?t=2415',
  'title': 'What are arrays, strings?'},
 {'url': 'https://youtu.be/FPu9Uld7W-E',
  'title': 'Time Complexity [Learn Basics, and then analyse in next Steps]'},
 {'url': 'https://www.youtube.com/watch?v=tNm_NNSB3_w&list=PLgUwDviBIf0oF6QL8m22w1hIDC1vJ_BHz&index=3',
  'title': 'Patterns'},
 {'url': 'https://www.youtube.com/watch?v=RRVYpIET_RU', 'title': 'C++ STL'},
 {'url': 'https://www.youtube.com/playlist?list=PLA3GkZPtsafZZsLj0Tybu3y0HVl-hp1ea',
  'title': 'Java Collections'},
 {'url': 'https://www.youtube.com/watch?v=KEs5UyBJ39g',
  'title': 'Hashing Theory'},
 {'url': 'https://www.geeksforgeeks.org/generate-binary-strings-without-consecutive-1s/?utm_source=youtube&utm_medium=collab_striver_ytdescription&utm_campaign=generate-binary-strings-without-consecutive-1s',
  'title': 'Generate all binary strings'},
 {'url': 'https://www.geeksforgeeks.org/find-minimum-number-of-coins-that-make-a-change/',
  'title': 'Greedy algorithm to find mini

In [45]:
# Expand YouTube urls
for item in tutorials_urls_titles:
    url = item['url']
    if "youtu.be" in url:
        item['url'] = expand_url(url)

In [46]:
tutorials_urls_titles

[{'url': 'https://www.youtube.com/watch?t=2415&v=EAR7De6Goz4&feature=youtu.be',
  'title': 'What are arrays, strings?'},
 {'url': 'https://www.youtube.com/watch?v=FPu9Uld7W-E&feature=youtu.be',
  'title': 'Time Complexity [Learn Basics, and then analyse in next Steps]'},
 {'url': 'https://www.youtube.com/watch?v=tNm_NNSB3_w&list=PLgUwDviBIf0oF6QL8m22w1hIDC1vJ_BHz&index=3',
  'title': 'Patterns'},
 {'url': 'https://www.youtube.com/watch?v=RRVYpIET_RU', 'title': 'C++ STL'},
 {'url': 'https://www.youtube.com/playlist?list=PLA3GkZPtsafZZsLj0Tybu3y0HVl-hp1ea',
  'title': 'Java Collections'},
 {'url': 'https://www.youtube.com/watch?v=KEs5UyBJ39g',
  'title': 'Hashing Theory'},
 {'url': 'https://www.geeksforgeeks.org/generate-binary-strings-without-consecutive-1s/?utm_source=youtube&utm_medium=collab_striver_ytdescription&utm_campaign=generate-binary-strings-without-consecutive-1s',
  'title': 'Generate all binary strings'},
 {'url': 'https://www.geeksforgeeks.org/find-minimum-number-of-coins

In [47]:
# Some more url standardising
tutorials_urls_titles[0]['url'] = "https://www.youtube.com/watch?v=EAR7De6Goz4"
tutorials_urls_titles[1]['url'] = "https://www.youtube.com/watch?v=FPu9Uld7W-E"
tutorials_urls_titles[2]['url'] = "https://www.youtube.com/watch?v=tNm_NNSB3_w"

In [49]:
print(tutorials_urls_titles[0]['url'])
print(tutorials_urls_titles[1]['url'])
print(tutorials_urls_titles[2]['url'])

https://www.youtube.com/watch?v=EAR7De6Goz4
https://www.youtube.com/watch?v=FPu9Uld7W-E
https://www.youtube.com/watch?v=tNm_NNSB3_w


In [20]:
urls = [item["url"] for item in problems_urls_titles]

In [21]:
len(urls)

447

In [22]:
urls[:5]

['https://practice.geeksforgeeks.org/problems/search-query-auto-complete/0',
 'https://www.geeksforgeeks.org/problems/data-type-1666706751/1',
 'https://practice.geeksforgeeks.org/problems/java-if-else-decision-making0924/0',
 'https://practice.geeksforgeeks.org/problems/java-switch-case-statement3529/1',
 'https://www.naukri.com/code360/problems/nth-fibonacci-number_74156']

In [36]:
parsed_data = problems_parse4db(urls, "striver-a2z-progress.json")
len(parsed_data)

Loaded 396 previously parsed items.


Processing URLs:  11%|█████▏                                       | 51/447 [00:42<05:27,  1.21it/s]


447

In [39]:
parsed_data[:3]

[{'id': '706686GFG',
  'type': 'problem',
  'title': 'Search Query Auto Complete',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/search-query-auto-complete/0'},
 {'id': '712535GFG',
  'type': 'problem',
  'title': 'Data Type',
  'platform': 'GFG',
  'href': 'https://www.geeksforgeeks.org/problems/data-type-1666706751/1'},
 {'id': '703776GFG',
  'type': 'problem',
  'title': 'Decision Making in Java',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/java-if-else-decision-making0924/0'}]

In [40]:
problems_parsed = [{**parsed_data[i], "title": problems_urls_titles[i]["title"]} for i in range(len(parsed_data))]

In [41]:
len(problems_parsed)

447

In [43]:
problems_parsed[:3]

[{'id': '706686GFG',
  'type': 'problem',
  'title': 'User Input / Output',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/search-query-auto-complete/0'},
 {'id': '712535GFG',
  'type': 'problem',
  'title': 'Data Types',
  'platform': 'GFG',
  'href': 'https://www.geeksforgeeks.org/problems/data-type-1666706751/1'},
 {'id': '703776GFG',
  'type': 'problem',
  'title': 'If Else statements',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/java-if-else-decision-making0924/0'}]

In [57]:
tutorials_parsed = tutorials_parse4db(tutorials_urls_titles)

Processing URLs: 100%|██████████████████████████████████████████████| 8/8 [00:00<00:00, 2877.74it/s]


In [58]:
tutorials_parsed

[{'id': '06e65f22-a381-534e-bdac-09e185b15f15YT',
  'type': 'tutorial',
  'title': 'What are arrays, strings?',
  'platform': 'YT',
  'href': 'https://www.youtube.com/watch?v=EAR7De6Goz4'},
 {'id': 'bfa5efb1-ae70-5806-96f6-a8d605a84317YT',
  'type': 'tutorial',
  'title': 'Time Complexity [Learn Basics, and then analyse in next Steps]',
  'platform': 'YT',
  'href': 'https://www.youtube.com/watch?v=FPu9Uld7W-E'},
 {'id': '5e2f1e1d-014e-5c63-b88d-a5be0df53444YT',
  'type': 'tutorial',
  'title': 'Patterns',
  'platform': 'YT',
  'href': 'https://www.youtube.com/watch?v=tNm_NNSB3_w'},
 {'id': '6291c4f9-96b9-54bc-9335-f8307ec9f525YT',
  'type': 'tutorial',
  'title': 'C++ STL',
  'platform': 'YT',
  'href': 'https://www.youtube.com/watch?v=RRVYpIET_RU'},
 {'id': '8783ff24-2988-5d04-afee-61eb5dafbe2eYT',
  'type': 'tutorial',
  'title': 'Java Collections',
  'platform': 'YT',
  'href': 'https://www.youtube.com/playlist?list=PLA3GkZPtsafZZsLj0Tybu3y0HVl-hp1ea'},
 {'id': '7553e515-d638-56ff-

In [59]:
parsed_merged = problems_parsed + tutorials_parsed

In [60]:
len(parsed_merged)

455

In [61]:
parsed_merged[:3]

[{'id': '706686GFG',
  'type': 'problem',
  'title': 'User Input / Output',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/search-query-auto-complete/0'},
 {'id': '712535GFG',
  'type': 'problem',
  'title': 'Data Types',
  'platform': 'GFG',
  'href': 'https://www.geeksforgeeks.org/problems/data-type-1666706751/1'},
 {'id': '703776GFG',
  'type': 'problem',
  'title': 'If Else statements',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/java-if-else-decision-making0924/0'}]

In [62]:
os.makedirs(os.path.dirname(export_file), exist_ok=True)

In [63]:
with open(export_file, "w", encoding="utf-8") as f:
    json.dump(parsed_merged, f, ensure_ascii=False, indent=4)